In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
import time

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
def get_box_face(img_path):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    out = detector.detect_faces(img)
    return out

# crop image and resize to 224x224
def crop_img(img_path, coords, out_path):
    image_obj = Image.open(img_path)
    cropped_image = image_obj.crop(coords)
    img = cropped_image.resize((224,224))
    img.save(out_path)
    
# bbox has 4 elements, 1st and 2nd elements are top left x,y coordinates
# 3rd and 4th elements are width and length
def show_img(img_path, bbox):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    cv_img = np.array(img)
    x1 = int(bbox[0])
    y1 = int(bbox[1])
    x2 = int(bbox[0]) + int(bbox[2])
    y2 = int(bbox[1]) + int(bbox[3])
    cv2.rectangle(cv_img, (x1, y1), (x2, y2), (0,255,0), 3)
    plt.imshow(cv_img)
    plt.show()
    
def rescale_box(box, scale):
    cx = box[0] + int(box[2]/2)
    cy = box[1] + int(box[3]/2)
    w_max = max(box[2], box[3])
    w = int(w_max * scale)
    x = cx - int(w/2)
    y = cy - int(w/2)
    box = [x,y,w,w]
    return box

# Test on single image

In [ ]:
img_path = "/home/luqman/ME-GraphAU/data/DISFA/img/SN009/1742.png"
box = get_box_face(img_path)
box

In [ ]:
show_img(img_path, box)

In [ ]:
s_box = rescale_box(box, 1.4)

In [ ]:
show_img(img_path, s_box)

In [ ]:
coords = (s_box[0], s_box[1], s_box[0]+s_box[2], s_box[1]+s_box[3])
out_path = "/home/luqman/img.png"
crop_img(img_path, coords, out_path)

# Run MTCNN

In [ ]:
start_time = time.time()
input_dir = "/home/luqman/ME-GraphAU/data/DISFA/img"
video_names = ['SN009']

for v in video_names:
    print(f'Processing {v}')
    boxes = []
    img_dir = os.path.join(input_dir, v)
    frame_count = len([frame for frame in os.listdir(img_dir)])
    
    for i in range(frame_count):
        filename = str(i+1) + '.png'
        img_path = os.path.join(img_dir, filename)
        box = get_box_face(img_path)
        boxes.append(box)

    out_filename = v + ".pickle"
    out_path = os.path.join('/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/', out_filename)
    outfile = open(out_path, 'wb')
    pickle.dump(boxes, outfile)
    outfile.close()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
(boxes[1742])

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Write new file

In [ ]:
f = open("log_crop_img.txt", "w")
f.write("All good!")
f.close()

# Convert video to frames

In [ ]:
start_time = time.time()
video_path = "/nas/database/DISFA_Database/Videos_LeftCamera/LeftVideoSN010_comp.avi"
out_path = "/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/SN010_frames"

command = "ffmpeg -i " + video_path + " " + os.path.join(out_path, "%d.png") 
os.system(command)
print("--- %s seconds ---" % (time.time() - start_time))